In [12]:
import cv2
import os
import numpy as np
import glob

def load_images_from_directory(directory):
    # Load images from a specified directory and return a list
    images = []
    for filename in sorted(glob.glob(os.path.join(directory, '*.png'))):  # Ensure this matches your file extension
        img = cv2.imread(filename)
        if img is not None:
            images.append(img)
            print(f"Loaded image: {filename}")  # Debugging print statement
        else:
            print(f"Failed to load image: {filename}")  # Debugging print statement
    print(f"Total images loaded from {directory}: {len(images)}")  # Debugging print statement
    return images

def resize_images(images, width=800):
    # Resize images to a specified width while maintaining aspect ratio
    resized_images = []
    for img in images:
        height = int((width / img.shape[1]) * img.shape[0])
        resized_images.append(cv2.resize(img, (width, height)))
    return resized_images

def stitch_pairwise(images):
    if len(images) < 2:
        print("Not enough images to stitch.")
        return None
    
    stitched = images[0]
    for i in range(1, len(images)):
        stitcher = cv2.Stitcher_create()
        status, stitched = stitcher.stitch((stitched, images[i]))
        if status != cv2.Stitcher_OK:
            print(f"Stitching failed between images {i-1} and {i} with status code: {status}")
            return None
    return stitched

def main():
    # Set directories
    stream1_directory = r'D:\Vortex\ROV\Phase 2\Tasks2025\task 5\stream 1'
    stream2_directory = r'D:\Vortex\ROV\Phase 2\Tasks2025\task 5\stream 2'
    
    # Load images from both streams
    images_stream1 = load_images_from_directory(stream1_directory)
    images_stream2 = load_images_from_directory(stream2_directory)

    # Check if images were loaded successfully
    if not images_stream1:
        print("No images loaded from stream 1. Exiting.")
        return
    if not images_stream2:
        print("No images loaded from stream 2. Exiting.")
        return

    # Resize images for better stitching
    images_stream1 = resize_images(images_stream1)
    images_stream2 = resize_images(images_stream2)

    # Stitch images from stream 1
    print("Stitching images from stream 1...")
    photomosaic_stream1 = stitch_pairwise(images_stream1)

    # Stitch images from stream 2
    print("Stitching images from stream 2...")
    photomosaic_stream2 = stitch_pairwise(images_stream2)

    # Check if stitching was successful and save the results
    if photomosaic_stream1 is not None:
        cv2.imwrite('photomosaic_stream1.jpg', photomosaic_stream1)
        print("Photomosaic for stream 1 saved successfully.")

    if photomosaic_stream2 is not None:
        cv2.imwrite('photomosaic_stream2.jpg', photomosaic_stream2)
        print("Photomosaic for stream 2 saved successfully.")

    # Display the results
    if photomosaic_stream1 is not None:
        cv2.imshow('Photomosaic Stream 1', photomosaic_stream1)
    if photomosaic_stream2 is not None:
        cv2.imshow('Photomosaic Stream 2', photomosaic_stream2)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Total images loaded from D:\Vortex\ROV\Phase 2\Tasks2025\task 5\stream 1: 0
Total images loaded from D:\Vortex\ROV\Phase 2\Tasks2025\task 5\stream 2: 0
No images loaded from stream 1. Exiting.


In [5]:
import cv2
import numpy as np
import os

def load_images_from_directory(directory):
    """Loads all images from a given directory, sorted by filename."""
    images = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
            else:
                print(f"Warning: Could not load image at {img_path}")
    return images

def resize_images(images, target_width, target_height):
    """Resizes all images to the specified width and height."""
    return [cv2.resize(img, (target_width, target_height)) for img in images]

def arrange_images(images):
    """Arranges images in a specific layout according to the reference image."""
    # Ensure we have the expected number of images
    if len(images) < 5:
        print("Error: At least 5 images are required for this layout.")
        return None

    # Define the target size for each individual image
    target_width = 200  # Adjust as needed
    target_height = 150  # Adjust as needed
    resized_images = resize_images(images, target_width, target_height)

    # Create a blank canvas with enough space for the layout
    canvas_height = target_height * 2 + 20  # extra space for separation
    canvas_width = target_width * 4 + 40  # extra space for separation
    canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255  # white background

    # Place the images on the canvas based on the desired layout
    # Top row (centered)
    x_offset_top = (canvas_width - target_width) // 2
    y_offset_top = 10  # padding from the top
    canvas[y_offset_top:y_offset_top+target_height, x_offset_top:x_offset_top+target_width] = resized_images[0]

    # Bottom row (four images side by side)
    y_offset_bottom = target_height + 20  # padding from the top row
    for i in range(1, 5):
        x_offset_bottom = 10 + (i - 1) * (target_width + 10)  # 10px padding between images
        canvas[y_offset_bottom:y_offset_bottom+target_height, x_offset_bottom:x_offset_bottom+target_width] = resized_images[i]

    return canvas

# Load and arrange images
directory = r"D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 5\stream 1"
images = load_images_from_directory(directory)

# Arrange images based on the new layout
final_layout = arrange_images(images)

# Display and save the final layout
if final_layout is not None:
    cv2.imshow("Final Layout", final_layout)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the arranged layout
    output_path = "final_layout_modified.png"
    cv2.imwrite(output_path, final_layout)
    print(f"Arranged layout saved as {output_path}")


Arranged layout saved as final_layout_modified.png


In [ ]:
import cv2
import numpy as np
import os

def load_images_from_directory(directory):
    """Loads all images from a given directory, sorted by filename."""
    images = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
            else:
                print(f"Warning: Could not load image at {img_path}")
    return images

def resize_images(images, target_width, target_height):
    """Resizes all images to the specified width and height."""
    return [cv2.resize(img, (target_width, target_height)) for img in images]

def arrange_images(images):
    """Arranges images in a specific layout according to the reference image."""
    # Ensure we have the expected number of images
    if len(images) < 5:
        print("Error: At least 5 images are required for this layout.")
        return None

    # Define the target size for each individual image
    target_width = 200  # Adjust as needed
    target_height = 150  # Adjust as needed
    resized_images = resize_images(images, target_width, target_height)

    # Create a blank canvas with enough space for the layout
    canvas_height = target_height * 2 + 20  # extra space for separation
    canvas_width = target_width * 4 + 40  # extra space for separation
    canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255  # white background

    # Place the images on the canvas based on the desired layout
    # Top row (centered)
    x_offset_top = (canvas_width - target_width) // 2
    y_offset_top = 10  # padding from the top
    canvas[y_offset_top:y_offset_top+target_height, x_offset_top:x_offset_top+target_width] = resized_images[0]

    # Bottom row (four images side by side)
    y_offset_bottom = target_height + 20  # padding from the top row
    for i in range(1, 5):
        x_offset_bottom = 10 + (i - 1) * (target_width + 10)  # 10px padding between images
        canvas[y_offset_bottom:y_offset_bottom+target_height, x_offset_bottom:x_offset_bottom+target_width] = resized_images[i]

    return canvas


In [6]:
# Load and arrange images
directory = r"D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 5\stream 1"
images = load_images_from_directory(directory)

# Arrange images based on the new layout
final_layout = arrange_images(images)

# Display and save the final layout
if final_layout is not None:
    cv2.imshow("Final Layout", final_layout)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the arranged layout
    output_path = r"D:\Vortex\ROV\SW-Training-Phase2-2025\Card_1\images of task 5\final_layout_modified.png"
    cv2.imwrite(output_path, final_layout)
    print(f"Arranged layout saved as {output_path}")

Arranged layout saved as D:\Vortex\ROV\SW-Training-Phase2-2025\Card_1\images of task 5\final_layout_modified.png


In [7]:
# Load and arrange images
directory2 = r"D:\Vortex\ROV\Phase 2\computer vision\Tasks2025\task 5\stream 2"
images2 = load_images_from_directory(directory2)

# Arrange images based on the new layout
final_layout_2= arrange_images(images)

# Display and save the final layout
if final_layout_2 is not None:
    cv2.imshow("Final Layout 2", final_layout_2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the arranged layout
    output_path = r"D:\Vortex\ROV\SW-Training-Phase2-2025\Card_1\images of task 5\final_layout_modified2.png"
    cv2.imwrite(output_path, final_layout_2)
    print(f"Arranged layout saved as {output_path}")

Arranged layout saved as D:\Vortex\ROV\SW-Training-Phase2-2025\Card_1\images of task 5\final_layout_modified2.png
